### IMPORTING LIBRARIES

Necessary ibraries are imported

In [67]:
import pandas as pd
from sklearn.compose import ColumnTransformer 
from sklearn.impute import SimpleImputer
import numpy as np
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.metrics import r2_score
from sklearn.ensemble import RandomForestRegressor
import plotly.express as px
import warnings
warnings.filterwarnings("ignore")
from sklearn.tree import DecisionTreeRegressor
import xgboost as XGBRegressor
from sklearn.metrics import mean_squared_error
from sklearn.feature_selection import SelectKBest, chi2, mutual_info_classif,f_regression
!pip install xgboost==1.0.1
import xgboost as xgb
import matplotlib.pyplot as plt

### Data Set Reading

In [2]:
df=pd.read_csv("D:DataSets/SCMS_Delivery_History_Dataset.csv")

In [3]:
pd.set_option("display.max_columns",100)
df.head()

,ID,Project Code,PQ #,PO / SO #,ASN/DN #,Country,Managed By,Fulfill Via,Vendor INCO Term,Shipment Mode,PQ First Sent to Client Date,PO Sent to Vendor Date,Scheduled Delivery Date,Delivered to Client Date,Delivery Recorded Date,Product Group,Sub Classification,Vendor,Item Description,Molecule/Test Type,Brand,Dosage,Dosage Form,Unit of Measure (Per Pack),Line Item Quantity,Line Item Value,Pack Price,Unit Price,Manufacturing Site,First Line Designation,Weight (Kilograms),Freight Cost (USD),Line Item Insurance (USD)
0,1,100-CI-T01,Pre-PQ Process,SCMS-4,ASN-8,Côte d'Ivoire,PMO - US,Direct Drop,EXW,Air,Pre-PQ Process,Date Not Captured,2-Jun-06,2-Jun-06,2-Jun-06,HRDT,HIV test,RANBAXY Fine Chemicals LTD.,"HIV, Reveal G3 Rapid HIV-1 Antibody Test, 30 T...","HIV, Reveal G3 Rapid HIV-1 Antibody Test",Reveal,NaN,Test kit,30,19,551.0,29.00,0.97,Ranbaxy Fine Chemicals LTD,Yes,13,780.34,NaN
1,3,108-VN-T01,Pre-PQ Process,SCMS-13,ASN-85,Vietnam,PMO - US,Direct Drop,EXW,Air,Pre-PQ Process,Date Not Captured,14-Nov-06,14-Nov-06,14-Nov-06,ARV,Pediatric,Aurobindo Pharma Limited,"Nevirapine 10mg/ml, oral suspension, Bottle, 2...",Nevirapine,Generic,10mg/ml,Oral suspension,240,1000,6200.0,6.20,0.03,"Aurobindo Unit III, India",Yes,358,4521.5,NaN
2,4,100-CI-T01,Pre-PQ Process,SCMS-20,ASN-14,Côte d'Ivoire,PMO - US,Direct Drop,FCA,Air,Pre-PQ Process,Date Not Captured,27-Aug-06,27-Aug-06,27-Aug-06,HRDT,HIV test,Abbott GmbH & Co. KG,"HIV 1/2, Determine Complete HIV Kit, 100 Tests","HIV 1/2, Determine Complete HIV Kit",Determine,NaN,Test kit,100,500,40000.0,80.00,0.80,ABBVIE GmbH & Co.KG Wiesbaden,Yes,171,1653.78,NaN
3,15,108-VN-T01,Pre-PQ Process,SCMS-78,ASN-50,Vietnam,PMO - US,Direct Drop,EXW,Air,Pre-PQ Process,Date Not Captured,1-Sep-06,1-Sep-06,1-Sep-06,ARV,Adult,SUN PHARMACEUTICAL INDUSTRIES LTD (RANBAXY LAB...,"Lamivudine 150mg, tablets, 60 Tabs",Lamivudine,Generic,150mg,Tablet,60,31920,127360.8,3.99,0.07,"Ranbaxy, Paonta Shahib, India",Yes,1855,16007.06,NaN
4,16,108-VN-T01,Pre-PQ Process,SCMS-81,ASN-55,Vietnam,PMO - US,Direct Drop,EXW,Air,Pre-PQ Process,Date Not Captured,11-Aug-06,11-Aug-06,11-Aug-06,ARV,Adult,Aurobindo Pharma Limited,"Stavudine 30mg, capsules, 60 Caps",Stavudine,Generic,30mg,Capsule,60,38000,121600.0,3.20,0.05,"Aurobindo Unit III, India",Yes,7590,45450.08,NaN


Describing Dataset 

In [4]:
df.describe()

,ID,Unit of Measure (Per Pack),Line Item Quantity,Line Item Value,Pack Price,Unit Price,Line Item Insurance (USD)
count,10324.000000,10324.000000,10324.000000,1.032400e+04,10324.000000,10324.000000,10037.000000
mean,51098.968229,77.990895,18332.534870,1.576506e+05,21.910241,0.611701,240.117626
std,31944.332496,76.579764,40035.302961,3.452921e+05,45.609223,3.275808,500.190568
min,1.000000,1.000000,1.000000,0.000000e+00,0.000000,0.000000,0.000000
25%,12795.750000,30.000000,408.000000,4.314593e+03,4.120000,0.080000,6.510000
50%,57540.500000,60.000000,3000.000000,3.047147e+04,9.300000,0.160000,47.040000
75%,83648.250000,90.000000,17039.750000,1.664471e+05,23.592500,0.470000,252.400000
max,86823.000000,1000.000000,619999.000000,5.951990e+06,1345.640000,238.650000,7708.440000


In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10324 entries, 0 to 10323
Data columns (total 33 columns):
 #   Column                        Non-Null Count  Dtype  
---  ------                        --------------  -----  
 0   ID                            10324 non-null  int64  
 1   Project Code                  10324 non-null  object 
 2   PQ #                          10324 non-null  object 
 3   PO / SO #                     10324 non-null  object 
 4   ASN/DN #                      10324 non-null  object 
 5   Country                       10324 non-null  object 
 6   Managed By                    10324 non-null  object 
 7   Fulfill Via                   10324 non-null  object 
 8   Vendor INCO Term              10324 non-null  object 
 9   Shipment Mode                 9964 non-null   object 
 10  PQ First Sent to Client Date  10324 non-null  object 
 11  PO Sent to Vendor Date        10324 non-null  object 
 12  Scheduled Delivery Date       10324 non-null  object 
 13  D

Since, out of 32 only 3 columns has the null values. 

### Pre-Processing

Now, since three types of date are given in data, its a possibility that pricing depends upon the day or month or year of delivery. Keeping this, the three dates are separated into days, months and years. 

Before that, its required to check that whether the three dates are same throughout the data or not. 

In [6]:
date_list=['Scheduled Delivery Date','Delivered to Client Date','Delivery Recorded Date']
same_date=[]
for i in range(len(date_list)-1):
    for j in df.index:
        if df[date_list[i]][j]!=df[date_list[i+1]][j]:
            same_date.append(j)

In [7]:
len(same_date)

5911

Since, the total length of rows is 10324 but only 5911 rows has same dates thus, possibility increases of target variable to be dependent on months or days or year. 

In [8]:
#Code to chnge the three days into days, month and years respectively. 

date_list=['Scheduled Delivery Date','Delivered to Client Date','Delivery Recorded Date']
df['Date_deliver_client']=np.nan
df['Date_Delivery']=np.nan
df['Date_deliver_record']=np.nan

target=['Date_deliver_client','Date_Delivery','Date_deliver_record']
def date_input(target,present):   
        for j in df.index:
            df[target][j]= df[present][j].split('-')[0] 
            
for i in range(0,3):
    date_input(target[i],date_list[i])

In [9]:
df['month_deliver_client']=np.nan
df['month_Delivery']=np.nan
df['month_deliver_record']=np.nan
target=['month_deliver_client','month_Delivery','month_deliver_record']
def month_input(target,present):   
        for j in df.index:
            df[target][j]= df[present][j].split('-')[1] 
for i in range(0,3):
    month_input(target[i],date_list[i])

In [10]:
df['year_deliver_client']=np.nan
df['year_Delivery']=np.nan
df['year_deliver_record']=np.nan
target=['year_deliver_client','year_Delivery','year_deliver_record']
def year_input(target,present):   
        for j in df.index:
            df[target][j]= df[present][j].split('-')[2] 
for i in range(0,3):
    year_input(target[i],date_list[i])

In [11]:
df.head()

,ID,Project Code,PQ #,PO / SO #,ASN/DN #,Country,Managed By,Fulfill Via,Vendor INCO Term,Shipment Mode,PQ First Sent to Client Date,PO Sent to Vendor Date,Scheduled Delivery Date,Delivered to Client Date,Delivery Recorded Date,Product Group,Sub Classification,Vendor,Item Description,Molecule/Test Type,Brand,Dosage,Dosage Form,Unit of Measure (Per Pack),Line Item Quantity,Line Item Value,Pack Price,Unit Price,Manufacturing Site,First Line Designation,Weight (Kilograms),Freight Cost (USD),Line Item Insurance (USD),Date_deliver_client,Date_Delivery,Date_deliver_record,month_deliver_client,month_Delivery,month_deliver_record,year_deliver_client,year_Delivery,year_deliver_record
0,1,100-CI-T01,Pre-PQ Process,SCMS-4,ASN-8,Côte d'Ivoire,PMO - US,Direct Drop,EXW,Air,Pre-PQ Process,Date Not Captured,2-Jun-06,2-Jun-06,2-Jun-06,HRDT,HIV test,RANBAXY Fine Chemicals LTD.,"HIV, Reveal G3 Rapid HIV-1 Antibody Test, 30 T...","HIV, Reveal G3 Rapid HIV-1 Antibody Test",Reveal,NaN,Test kit,30,19,551.0,29.00,0.97,Ranbaxy Fine Chemicals LTD,Yes,13,780.34,NaN,2.0,2.0,2.0,Jun,Jun,Jun,6.0,6.0,6.0
1,3,108-VN-T01,Pre-PQ Process,SCMS-13,ASN-85,Vietnam,PMO - US,Direct Drop,EXW,Air,Pre-PQ Process,Date Not Captured,14-Nov-06,14-Nov-06,14-Nov-06,ARV,Pediatric,Aurobindo Pharma Limited,"Nevirapine 10mg/ml, oral suspension, Bottle, 2...",Nevirapine,Generic,10mg/ml,Oral suspension,240,1000,6200.0,6.20,0.03,"Aurobindo Unit III, India",Yes,358,4521.5,NaN,14.0,14.0,14.0,Nov,Nov,Nov,6.0,6.0,6.0
2,4,100-CI-T01,Pre-PQ Process,SCMS-20,ASN-14,Côte d'Ivoire,PMO - US,Direct Drop,FCA,Air,Pre-PQ Process,Date Not Captured,27-Aug-06,27-Aug-06,27-Aug-06,HRDT,HIV test,Abbott GmbH & Co. KG,"HIV 1/2, Determine Complete HIV Kit, 100 Tests","HIV 1/2, Determine Complete HIV Kit",Determine,NaN,Test kit,100,500,40000.0,80.00,0.80,ABBVIE GmbH & Co.KG Wiesbaden,Yes,171,1653.78,NaN,27.0,27.0,27.0,Aug,Aug,Aug,6.0,6.0,6.0
3,15,108-VN-T01,Pre-PQ Process,SCMS-78,ASN-50,Vietnam,PMO - US,Direct Drop,EXW,Air,Pre-PQ Process,Date Not Captured,1-Sep-06,1-Sep-06,1-Sep-06,ARV,Adult,SUN PHARMACEUTICAL INDUSTRIES LTD (RANBAXY LAB...,"Lamivudine 150mg, tablets, 60 Tabs",Lamivudine,Generic,150mg,Tablet,60,31920,127360.8,3.99,0.07,"Ranbaxy, Paonta Shahib, India",Yes,1855,16007.06,NaN,1.0,1.0,1.0,Sep,Sep,Sep,6.0,6.0,6.0
4,16,108-VN-T01,Pre-PQ Process,SCMS-81,ASN-55,Vietnam,PMO - US,Direct Drop,EXW,Air,Pre-PQ Process,Date Not Captured,11-Aug-06,11-Aug-06,11-Aug-06,ARV,Adult,Aurobindo Pharma Limited,"Stavudine 30mg, capsules, 60 Caps",Stavudine,Generic,30mg,Capsule,60,38000,121600.0,3.20,0.05,"Aurobindo Unit III, India",Yes,7590,45450.08,NaN,11.0,11.0,11.0,Aug,Aug,Aug,6.0,6.0,6.0


Since, the date list is not required anymore in dataframe, its drop out

In [12]:
df.drop(date_list,axis=1,inplace=True)

In [13]:
df.dtypes

ID                                int64
Project Code                     object
PQ #                             object
PO / SO #                        object
ASN/DN #                         object
Country                          object
Managed By                       object
Fulfill Via                      object
Vendor INCO Term                 object
Shipment Mode                    object
PQ First Sent to Client Date     object
PO Sent to Vendor Date           object
Product Group                    object
Sub Classification               object
Vendor                           object
Item Description                 object
Molecule/Test Type               object
Brand                            object
Dosage                           object
Dosage Form                      object
Unit of Measure (Per Pack)        int64
Line Item Quantity                int64
Line Item Value                 float64
Pack Price                      float64
Unit Price                      float64


As seen above, its known that weight and freight cost columns should be int or float but the described dtype is object, thus processing of weight column is done. 

In [15]:
wt_list=df['Weight (Kilograms)'].tolist()
wt_new={}
for i in range(len(wt_list)):
    try:
        pd.to_numeric(wt_list[i])
            
    except:
        wt_new[i]=wt_list[i]
len(wt_new)
df['wt_cat']=np.nan

In [16]:
for i in wt_new.keys():
    df['wt_cat'][i]=wt_new[i]
    df['Weight (Kilograms)'][i]=0

In [17]:
df['Weight (Kilograms)']=df['Weight (Kilograms)'].astype(int)

Similarly processing of freight cost column is done. The categorical values in both the columns are replaced with 0 and new column is formed with categorical values. 

In [18]:
ft_list=df['Freight Cost (USD)'].tolist()
ft_new={}
for i in range(len(ft_list)):
    try:
        pd.to_numeric(ft_list[i])
            
    except:
        ft_new[i]=ft_list[i]
len(ft_new)
df['ft_cat']=np.nan

In [19]:
for i in ft_new.keys():
    df['ft_cat'][i]=ft_new[i]
    df['Freight Cost (USD)'][i]=0

In [20]:
df['Freight Cost (USD)']=df['Freight Cost (USD)'].astype(float)

After changing the columns data type to float, the dataset is fine now. 

In [21]:
df.dtypes

ID                                int64
Project Code                     object
PQ #                             object
PO / SO #                        object
ASN/DN #                         object
Country                          object
Managed By                       object
Fulfill Via                      object
Vendor INCO Term                 object
Shipment Mode                    object
PQ First Sent to Client Date     object
PO Sent to Vendor Date           object
Product Group                    object
Sub Classification               object
Vendor                           object
Item Description                 object
Molecule/Test Type               object
Brand                            object
Dosage                           object
Dosage Form                      object
Unit of Measure (Per Pack)        int64
Line Item Quantity                int64
Line Item Value                 float64
Pack Price                      float64
Unit Price                      float64


Filling the null values with -1 and 0 depending upon the requirement and data type of columns

In [22]:
df['Dosage Form'].fillna(-1,inplace=True)
df['Line Item Insurance (USD)'].fillna(0,inplace=True)

In [23]:
df.to_excel("C:final_sheel.xlsx")

### VISUALIZATION

Following Questions are being answered during the visualization. 

#### Q1. Which Country and Brand has the maximum Unit Price ?

In [56]:
print("The sum of Generic Unit Price total is ", df[df['Brand']=='Generic']['Unit Price'].sum())
px.bar(df.groupby('Brand')['Unit Price'].count(), title='Count of Unit Prices wrt Brands')

The sum of Generic Unit Price total is  1287.3200000000002


#### As shown Generic Brand has the maximum total count of unit price i.e. 7285 with the total sum of 1287.32. This means that count of shipping of Generic brand is more than any other brand

In [129]:
px.bar(df.groupby('Brand')['Unit Price'].sum(), title='Sum of Unit Prices wrt Brands',color_discrete_sequence =['red']*3)

#### Though as explained in above graph, the maximum cost of unit price shipped is for brand Colloidal Gold, though count is maximum for Generic which comes at second number in total cost of unit price shipping. 

#### This shows that Colloidal Brand has shipped maximum cost in just 70 counts whereas, generic has shipped only 0.7633% of colloidal in 7285 counts. Seems, genric brand is in good growing phase where customer trails the product in small packets. 

#### Other brand which is growing phase is Determines which sold 791.67 in 799 counts. Seems like, the growing phase of Determines is better than generic though the selling cost is less, because the ratio of count and selling cost is more of determines than generic. 

In [128]:
px.bar(df.groupby('Country')['Unit Price'].sum()/df.groupby('Country')['Unit Price'].count(), title='Ratio of Sum to Count of Unit Prices wrt Country',color_discrete_sequence =['green']*3)

#### The above graph is displaying the ratio of sum and count. The maximum ratio is of Mali i.e. 6.10 means the maximum shipping cost of unit price is done for Mali country with minimum no. of shipping times. Opposite to this, Country Burkina Faso has the ratio 0.09 means maximum count of shipping but still cost of shipping is least. 

#### The country Mali is said to have maximum cost in export followed by Ethiopia, Botswana and Kazakhstan. 

#### If the ratio is more means the sum of export of unit price is more than the number of times the export has been done from particular country.

#### Q2 Which Dosage Form has maximum Pack Price or maximum shipped in terms of price ?

In [135]:
px.bar(df.groupby('Dosage Form')['Pack Price'].sum(), title='Sum of Pack Prices wrt Vendor',color_discrete_sequence =['black']*3)

#### Maximum Pack Price of Test Kit with 115k cost is shipped followed by Tablet with 50.03k cost. This implies may be that citizens prefer to go for test kit and tablet rather than any other form of dosage. 

#### But this also means that may be test kits are more costly than any other form. It can be possible that supply of test kit is least but since may be test kit is costly thats why the bar of test kit is highest.

### MODEL INITIALIZATION

In [24]:
X=df.drop(['Pack Price','Unit Price'],axis=1)

Since, the target variable is unit price and pack price therefore, two Y are taken.

In [25]:
y=df['Pack Price']
Y=df['Unit Price']

Application of Transformers

In [26]:
cat_col=[]
num_col=[]
for i in X.columns:
    if X[i].dtype=='O':
        cat_col.append(i)
    else:
        num_col.append(i)
categorical_transformer = OneHotEncoder(handle_unknown='ignore')

In [137]:
import pickle
pickle.dump(categorical_transformer, open('cattrans.pkl','wb'))

In [27]:
numeric_transformer = Pipeline(steps=[('scaler',MinMaxScaler()) ])

In [138]:
pickle.dump(numeric_transformer, open('numerictrans.pkl','wb'))

In [28]:
preprocessing=ColumnTransformer(remainder='passthrough', transformers=[('cat', categorical_transformer, cat_col), ('num',numeric_transformer,num_col)])

In [139]:
pickle.dump(preprocessing, open('prepro.pkl','wb'))

FOR UNIT PRICE 

In [29]:
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.2,
                                                    random_state=0)

In [34]:
final_list=[]
model=[LinearRegression(),DecisionTreeRegressor(),RandomForestRegressor(),xgb.XGBRegressor()]
result={}
for j in  [5, 10, 15, 20, '25', '30','35','39']:
        new=[]
        for i in model:
            
            runpipe = Pipeline(steps=[('preprocessor', preprocessing), ('skb',SelectKBest(score_func=f_regression, k = int(j))),
                              ('model', i)])
            runpipe.fit(X_train,Y_train)
            pred=runpipe.predict(X_test)
            rmse = np.sqrt(mean_squared_error(Y_test, pred))
            new.append(runpipe.score(X_test, Y_test))
        final_list.append(new)

In [35]:
pd.DataFrame(final_list,index=['5', '10', '15', '20', '25', '30','35','39'], columns=['Linear Regression','Decision Tree Regressor','Random Forest Regressor','XGBRegressor()'])

,Linear Regression,Decision Tree Regressor,Random Forest Regressor,XGBRegressor()
5,-0.002967,-0.002967,-0.003144,-0.002967
10,0.731158,0.731158,0.731144,0.731158
15,0.816197,0.816197,0.815816,0.816198
20,0.820665,0.824756,0.824752,0.824757
25,0.820532,0.824191,0.824214,0.824193
30,0.821485,0.825144,0.825796,0.825166
35,0.891285,0.895023,0.894384,0.895026
39,0.890749,0.852023,0.860117,0.851924


As given table describes the maximum accuracy is 0.8950 with XGB Regressor with 35 features for prediction of UNIT PRICE

FOR PACK PRICE

In [36]:
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.2,
                                                    random_state=0)

In [37]:
final_list=[]
model=[LinearRegression(),DecisionTreeRegressor(),RandomForestRegressor(),xgb.XGBRegressor()]
result={}
for j in [5, 10, '15', '20', '25', '30','35','39']:
        new=[]
        for i in model:
            
            runpipe = Pipeline(steps=[('preprocessor', preprocessing), ('skb',SelectKBest(score_func=f_regression, k = int(j))),
                              ('model', i)])
            runpipe.fit(X_train,y_train)
            pred=runpipe.predict(X_test)
            rmse = np.sqrt(mean_squared_error(y_test, pred))
            new.append(runpipe.score(X_test, y_test))
        final_list.append(new)

In [38]:
pd.DataFrame(final_list,index=['5', '10', '15', '20', '25', '30','35','39'], columns=['Linear Regression','Decision Tree Regressor','Random Forest Regressor','XGBRegressor()'])

,Linear Regression,Decision Tree Regressor,Random Forest Regressor,XGBRegressor()
5,-0.002967,-0.002967,-0.003321,-0.002967
10,0.731158,0.731158,0.731126,0.731158
15,0.816197,0.816197,0.815781,0.816198
20,0.820665,0.824756,0.824933,0.824757
25,0.820532,0.824191,0.823740,0.824193
30,0.821485,0.825144,0.824881,0.825166
35,0.891285,0.895023,0.894668,0.895026
39,0.890749,0.852023,0.853415,0.851924


As given table describes the maximum accuracy is 0.8950 with XGB Regressor with 35 features for prediction of UNIT PRICE

### HYPERTUNING OF RANDOM FOREST 

In [39]:
pipeline = Pipeline(steps=[('preprocessor', preprocessing), ('skb',SelectKBest(score_func=f_regression, k = int(j))),
                              ('model', RandomForestRegressor())])
parameters={'model__max_depth':[50,None],
    'model__min_samples_split':[2,5],
    'model__min_samples_leaf':[1,2]}

In [140]:
pickle.dump(preprocessing, open('prepro.pkl','wb'))

In [40]:
CV = GridSearchCV(pipeline, parameters, scoring = 'neg_mean_absolute_error', n_jobs= 1)
CV.fit(X_train, y_train)  

GridSearchCV(estimator=Pipeline(steps=[('preprocessor',
                                        ColumnTransformer(remainder='passthrough',
                                                          transformers=[('cat',
                                                                         OneHotEncoder(handle_unknown='ignore'),
                                                                         ['Project '
                                                                          'Code',
                                                                          'PQ '
                                                                          '#',
                                                                          'PO '
                                                                          '/ '
                                                                          'SO '
                                                                          '#',
                        

In [41]:
CV.best_params_

{'model__max_depth': 50,
 'model__min_samples_leaf': 1,
 'model__min_samples_split': 5}

In [42]:
y_pred = CV.predict(X_test)
r2_score(y_test,y_pred)

0.8564169810470692

Since, after hypertuning of Random Forest, maximum accuracy achieved is 0.856, there fore, go head with 0.89